# Simulating long experiments

This notebook introduces functionality for simulating experiments over hundreds or even thousands of cycles. 

In the interest of simplicity and running time, we consider a SPM with SEI effects leading to linear degradation.

- termination
- save_at_cycles
- summary variables
- starting solution

In [1]:
%pip install pybamm -q
import pybamm
import matplotlib.pyplot as plt
import numpy as np

You should consider upgrading via the '/Users/vsulzer/Documents/Energy_storage/PyBaMM/.tox/dev/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)
parameter_values.update(
    {
        "SEI kinetic rate constant [m.s-1]": 1e-15,
#         "SEI resistivity [Ohm.m]": 0,
    },
)
spm = pybamm.lithium_ion.SPM({"SEI": "ec reaction limited"})


In [3]:
esoh_model = pybamm.lithium_ion.ElectrodeSOH()
esoh_sim = pybamm.Simulation(esoh_model, parameter_values=parameter_values)
param = spm.param

Vmin = 3.0
Vmax = 4.2
Cn = parameter_values.evaluate(param.C_n_init)
Cp = parameter_values.evaluate(param.C_p_init)
n_Li_init = parameter_values.evaluate(param.n_Li_particles_init)
c_n_max = parameter_values.evaluate(param.c_n_max)
c_p_max = parameter_values.evaluate(param.c_p_max)

esoh_sol = esoh_sim.solve(
    [0],
    inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li_init},
)
print(esoh_sol["x_100"].data[0])
print(esoh_sol["y_100"].data[0])

0.8334162315444245
0.03350230659015597


In [4]:
pybamm.set_logging_level("NOTICE")
parameter_values.update(
    {
        "Initial concentration in negative electrode [mol.m-3]": esoh_sol["x_100"].data[0] * c_n_max,
        "Initial concentration in positive electrode [mol.m-3]": esoh_sol["y_100"].data[0] * c_p_max,
    }
)
experiment = pybamm.Experiment([
    (f"Discharge at 1C until {Vmin}V",
     "Rest for 1 hour",
     f"Charge at 1C until {Vmax}V", 
     f"Hold at {Vmax}V until C/50"
    )
])
sim = pybamm.Simulation(spm, experiment=experiment, parameter_values=parameter_values, 
                        solver=pybamm.CasadiSolver("safe"))
spm_sol = sim.solve()

2021-05-17 16:46:48,672 - [NOTICE] simulation.solve(710): Cycle 1/1 (116.121 ms elapsed) --------------------
2021-05-17 16:46:48,673 - [NOTICE] simulation.solve(742): Cycle 1/1, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:46:48,769 - [NOTICE] simulation.solve(742): Cycle 1/1, step 2/4: Rest for 1 hour
2021-05-17 16:46:48,841 - [NOTICE] simulation.solve(742): Cycle 1/1, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:46:48,935 - [NOTICE] simulation.solve(742): Cycle 1/1, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:46:49,277 - [NOTICE] simulation.solve(837): Finish experiment simulation, took 721.036 ms


In [5]:
pybamm.set_logging_level("NOTICE")
experiment = pybamm.Experiment([
    (f"Discharge at 1C until {Vmin}V",
     "Rest for 1 hour",
    f"Charge at 1C until {Vmax}V", 
    f"Hold at {Vmax}V until C/50")
] * 500,
termination="80% capacity"
)
sim_100 = pybamm.Simulation(spm, experiment=experiment, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
spm_sol_100 = sim_100.solve()

2021-05-17 16:46:50,972 - [NOTICE] simulation.solve(710): Cycle 1/500 (34.271 ms elapsed) --------------------
2021-05-17 16:46:50,972 - [NOTICE] simulation.solve(742): Cycle 1/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:46:51,061 - [NOTICE] simulation.solve(742): Cycle 1/500, step 2/4: Rest for 1 hour
2021-05-17 16:46:51,137 - [NOTICE] simulation.solve(742): Cycle 1/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:46:51,230 - [NOTICE] simulation.solve(742): Cycle 1/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:46:51,565 - [NOTICE] simulation.solve(820): Capacity is now 4.966 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:46:51,566 - [NOTICE] simulation.solve(710): Cycle 2/500 (628.372 ms elapsed) --------------------
2021-05-17 16:46:51,566 - [NOTICE] simulation.solve(742): Cycle 2/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:46:51,614 - [NOTICE] simulation.solve(742): Cycle 2/500, step 2/4: Rest for 1 hour
2021-05-17 16:46:51,650 - [NO

2021-05-17 16:46:53,910 - [NOTICE] simulation.solve(742): Cycle 13/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:46:53,967 - [NOTICE] simulation.solve(820): Capacity is now 4.932 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:46:53,968 - [NOTICE] simulation.solve(710): Cycle 14/500 (3.030 s elapsed) --------------------
2021-05-17 16:46:53,968 - [NOTICE] simulation.solve(742): Cycle 14/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:46:54,014 - [NOTICE] simulation.solve(742): Cycle 14/500, step 2/4: Rest for 1 hour
2021-05-17 16:46:54,047 - [NOTICE] simulation.solve(742): Cycle 14/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:46:54,088 - [NOTICE] simulation.solve(742): Cycle 14/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:46:54,152 - [NOTICE] simulation.solve(820): Capacity is now 4.929 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:46:54,152 - [NOTICE] simulation.solve(710): Cycle 15/500 (3.215 s elapsed) --------------------

2021-05-17 16:46:56,251 - [NOTICE] simulation.solve(742): Cycle 26/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:46:56,297 - [NOTICE] simulation.solve(742): Cycle 26/500, step 2/4: Rest for 1 hour
2021-05-17 16:46:56,328 - [NOTICE] simulation.solve(742): Cycle 26/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:46:56,367 - [NOTICE] simulation.solve(742): Cycle 26/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:46:56,423 - [NOTICE] simulation.solve(820): Capacity is now 4.895 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:46:56,424 - [NOTICE] simulation.solve(710): Cycle 27/500 (5.486 s elapsed) --------------------
2021-05-17 16:46:56,424 - [NOTICE] simulation.solve(742): Cycle 27/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:46:56,471 - [NOTICE] simulation.solve(742): Cycle 27/500, step 2/4: Rest for 1 hour
2021-05-17 16:46:56,502 - [NOTICE] simulation.solve(742): Cycle 27/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:46:56,546 - [NOT

2021-05-17 16:46:58,612 - [NOTICE] simulation.solve(742): Cycle 38/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:46:58,671 - [NOTICE] simulation.solve(820): Capacity is now 4.861 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:46:58,671 - [NOTICE] simulation.solve(710): Cycle 39/500 (7.733 s elapsed) --------------------
2021-05-17 16:46:58,672 - [NOTICE] simulation.solve(742): Cycle 39/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:46:58,715 - [NOTICE] simulation.solve(742): Cycle 39/500, step 2/4: Rest for 1 hour
2021-05-17 16:46:58,748 - [NOTICE] simulation.solve(742): Cycle 39/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:46:58,798 - [NOTICE] simulation.solve(742): Cycle 39/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:46:58,860 - [NOTICE] simulation.solve(820): Capacity is now 4.859 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:46:58,861 - [NOTICE] simulation.solve(710): Cycle 40/500 (7.923 s elapsed) --------------------

2021-05-17 16:47:00,991 - [NOTICE] simulation.solve(742): Cycle 51/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:47:01,045 - [NOTICE] simulation.solve(742): Cycle 51/500, step 2/4: Rest for 1 hour
2021-05-17 16:47:01,080 - [NOTICE] simulation.solve(742): Cycle 51/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:47:01,255 - [NOTICE] simulation.solve(742): Cycle 51/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:47:01,333 - [NOTICE] simulation.solve(820): Capacity is now 4.825 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:47:01,334 - [NOTICE] simulation.solve(710): Cycle 52/500 (10.396 s elapsed) --------------------
2021-05-17 16:47:01,334 - [NOTICE] simulation.solve(742): Cycle 52/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:47:01,383 - [NOTICE] simulation.solve(742): Cycle 52/500, step 2/4: Rest for 1 hour
2021-05-17 16:47:01,421 - [NOTICE] simulation.solve(742): Cycle 52/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:47:01,464 - [NO

...

2021-05-17 16:48:27,155 - [NOTICE] simulation.solve(742): Cycle 448/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:48:27,199 - [NOTICE] simulation.solve(742): Cycle 448/500, step 2/4: Rest for 1 hour
2021-05-17 16:48:27,237 - [NOTICE] simulation.solve(742): Cycle 448/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:48:27,281 - [NOTICE] simulation.solve(742): Cycle 448/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:48:27,365 - [NOTICE] simulation.solve(820): Capacity is now nan Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:48:27,366 - [NOTICE] simulation.solve(710): Cycle 449/500 (1 minute, 36 seconds elapsed) --------------------
2021-05-17 16:48:27,366 - [NOTICE] simulation.solve(742): Cycle 449/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:48:27,422 - [NOTICE] simulation.solve(742): Cycle 449/500, step 2/4: Rest for 1 hour
2021-05-17 16:48:27,461 - [NOTICE] simulation.solve(742): Cycle 449/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 

2021-05-17 16:48:29,866 - [NOTICE] simulation.solve(742): Cycle 460/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:48:29,913 - [NOTICE] simulation.solve(742): Cycle 460/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:48:30,004 - [NOTICE] simulation.solve(820): Capacity is now nan Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:48:30,005 - [NOTICE] simulation.solve(710): Cycle 461/500 (1 minute, 39 seconds elapsed) --------------------
2021-05-17 16:48:30,005 - [NOTICE] simulation.solve(742): Cycle 461/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:48:30,057 - [NOTICE] simulation.solve(742): Cycle 461/500, step 2/4: Rest for 1 hour
2021-05-17 16:48:30,111 - [NOTICE] simulation.solve(742): Cycle 461/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:48:30,158 - [NOTICE] simulation.solve(742): Cycle 461/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:48:30,245 - [NOTICE] simulation.solve(820): Capacity is now nan Ah (originally 4.966 Ah, will stop 

2021-05-17 16:48:32,780 - [NOTICE] simulation.solve(820): Capacity is now nan Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:48:32,781 - [NOTICE] simulation.solve(710): Cycle 473/500 (1 minute, 42 seconds elapsed) --------------------
2021-05-17 16:48:32,781 - [NOTICE] simulation.solve(742): Cycle 473/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:48:32,826 - [NOTICE] simulation.solve(742): Cycle 473/500, step 2/4: Rest for 1 hour
2021-05-17 16:48:32,864 - [NOTICE] simulation.solve(742): Cycle 473/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:48:32,908 - [NOTICE] simulation.solve(742): Cycle 473/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:48:32,994 - [NOTICE] simulation.solve(820): Capacity is now nan Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:48:32,994 - [NOTICE] simulation.solve(710): Cycle 474/500 (1 minute, 42 seconds elapsed) --------------------
2021-05-17 16:48:32,995 - [NOTICE] simulation.solve(742): Cycle 474/500, step

2021-05-17 16:48:35,374 - [NOTICE] simulation.solve(710): Cycle 485/500 (1 minute, 44 seconds elapsed) --------------------
2021-05-17 16:48:35,375 - [NOTICE] simulation.solve(742): Cycle 485/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:48:35,426 - [NOTICE] simulation.solve(742): Cycle 485/500, step 2/4: Rest for 1 hour
2021-05-17 16:48:35,469 - [NOTICE] simulation.solve(742): Cycle 485/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:48:35,507 - [NOTICE] simulation.solve(742): Cycle 485/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:48:35,592 - [NOTICE] simulation.solve(820): Capacity is now nan Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:48:35,592 - [NOTICE] simulation.solve(710): Cycle 486/500 (1 minute, 45 seconds elapsed) --------------------
2021-05-17 16:48:35,593 - [NOTICE] simulation.solve(742): Cycle 486/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:48:35,637 - [NOTICE] simulation.solve(742): Cycle 486/500, step 2/4: Rest for 1

2021-05-17 16:48:38,121 - [NOTICE] simulation.solve(742): Cycle 497/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:48:38,170 - [NOTICE] simulation.solve(742): Cycle 497/500, step 2/4: Rest for 1 hour
2021-05-17 16:48:38,210 - [NOTICE] simulation.solve(742): Cycle 497/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 16:48:38,248 - [NOTICE] simulation.solve(742): Cycle 497/500, step 4/4: Hold at 4.2V until C/50
2021-05-17 16:48:38,334 - [NOTICE] simulation.solve(820): Capacity is now nan Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-17 16:48:38,335 - [NOTICE] simulation.solve(710): Cycle 498/500 (1 minute, 47 seconds elapsed) --------------------
2021-05-17 16:48:38,336 - [NOTICE] simulation.solve(742): Cycle 498/500, step 1/4: Discharge at 1C until 3.0V
2021-05-17 16:48:38,386 - [NOTICE] simulation.solve(742): Cycle 498/500, step 2/4: Rest for 1 hour
2021-05-17 16:48:38,440 - [NOTICE] simulation.solve(742): Cycle 498/500, step 3/4: Charge at 1C until 4.2V
2021-05-17 

In [9]:
spm_sol_100.summary_variables["Total lithium in particles [mol]"]

array([0.19287892, 0.1927693 , 0.1926598 , 0.19255039, 0.19244109,
       0.19233189, 0.19222279, 0.19211379, 0.1920049 , 0.19189611,
       0.19178742, 0.19167882, 0.19157034, 0.19146196, 0.19135368,
       0.1912455 , 0.19113743, 0.19102946, 0.19092158, 0.19081381,
       0.19070615, 0.19059858, 0.19049111, 0.19038375, 0.19027649,
       0.19016932, 0.19006225, 0.18995527, 0.1898484 , 0.18974163,
       0.18963496, 0.18952839, 0.18942192, 0.18931554, 0.18920927,
       0.18910309, 0.18899701, 0.18889102, 0.18878512, 0.18867933,
       0.18857363, 0.18846803, 0.18836252, 0.18825712, 0.18815181,
       0.1880466 , 0.18794149, 0.18783648, 0.18773157, 0.18762675,
       0.18752203, 0.18741741, 0.18731287, 0.18720842, 0.18710407,
       0.18699982, 0.18689566, 0.1867916 , 0.18668764, 0.18658377,
       0.18648   , 0.18637632, 0.18627274, 0.18616926, 0.18606587,
       0.18596258, 0.18585938, 0.18575627, 0.18565326, 0.18555035,
       0.18544753, 0.1853448 , 0.18524216, 0.18513962, 0.18503

## References

The relevant papers for this notebook are:

In [7]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[3] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/2.0341915jes.
[4] Peyman Mohtat, Suhak Lee, Jason B Siegel, and Anna G Stefanopoulou. Towards better estimability of electrode-specific state of health: decoding the cell expansion. Journal of Power Sources, 427:101–111, 201